# Comparação de SVM com Diferentes Kernels e Datasets

Este notebook demonstra como diferentes **kernels** de SVM (Support Vector Machine) se comportam em **datasets com diferentes tipos de fronteira de decisão**.

## Objetivos
- Importar bibliotecas necessárias
- Gerar datasets sintéticos com características distintas
- Dividir em **base de treino** e **base de teste**
- Padronizar os dados
- Treinar modelos `SVC` (com parâmetro **C**) e `NuSVC` (com parâmetro **nu**)
- Avaliar métricas de acurácia
- Plotar **fronteiras de decisão** e **matrizes de confusão**


# SVM: adequação de *kernel* ao tipo de fronteira de decisão

Este notebook gera **três datasets** com características distintas e avalia **SVC (C)** e **NuSVC (nu)** com três *kernels* (`linear`, `poly` com grau 2 e `rbf`).

- **Dataset "linear"**: classes aproximadamente separáveis por **hiperplano** — favorece `linear`.
- **Dataset "polynomial (circles)"**: **círculos concêntricos** (fronteira quadrática \(x^2+y^2=r^2\)) — `poly` (grau 2) se ajusta bem; `rbf` também costuma ir bem.
- **Dataset "rbf (moons)"**: **moons** com curvaturas locais — `rbf` tende a dominar.

Para cada dataset, treinamos os 6 modelos (SVC/NuSVC × 3 kernels) e reportamos **accuracy** e **balanced accuracy**, além de **matrizes de confusão** e **fronteiras de decisão** em 2D.


In [ ]:
# Importação das bibliotecas necessárias
import numpy as np
# Importação das bibliotecas necessárias
import matplotlib.pyplot as plt
# Importação das bibliotecas necessárias
from sklearn.datasets import make_classification, make_circles, make_moons
# Importação das bibliotecas necessárias
from sklearn.model_selection import train_test_split
# Importação das bibliotecas necessárias
from sklearn.preprocessing import StandardScaler
# Importação das bibliotecas necessárias
from sklearn.svm import SVC, NuSVC
# Importação das bibliotecas necessárias
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
# Importação das bibliotecas necessárias
import pandas as pd

def build_linear(n=600, random_state=7):
# Geração de dataset sintético
    X, y = make_classification(n_samples=n, n_features=2, n_informative=2, n_redundant=0,
                               n_clusters_per_class=1, class_sep=2.0, flip_y=0.01, random_state=random_state)
    return X, y

def build_poly(n=600, random_state=7):
# Geração de dataset sintético
    X, y = make_circles(n_samples=n, factor=0.45, noise=0.08, random_state=random_state)
    return X, y

def build_rbf(n=600, random_state=7):
# Geração de dataset sintético
    X, y = make_moons(n_samples=n, noise=0.15, random_state=random_state)
    return X, y

datasets = {
    "linear": build_linear(),
    "polynomial (circles)": build_poly(),
    "rbf (moons)": build_rbf()
}

def split_scale(X, y, test_size=0.3, seed=37):
# Divisão em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=seed)
# Padronização das variáveis (média 0, desvio padrão 1)
    sc = StandardScaler()
    X_train_s = sc.fit_transform(X_train)
    X_test_s = sc.transform(X_test)
    return X_train_s, X_test_s, y_train, y_test

def eval_models(X_train, X_test, y_train, y_test, C=1.0, nu=0.3, degree=2, gamma="scale"):
    results = []
    models = {}
    cfgs = [
# Criação e treino do modelo SVM com kernel especificado
        ("SVC", "linear", dict(kernel="linear", C=C)),
# Criação e treino do modelo SVM com kernel especificado
        ("SVC", "poly",   dict(kernel="poly", degree=degree, C=C, gamma=gamma)),
# Criação e treino do modelo SVM com kernel especificado
        ("SVC", "rbf",    dict(kernel="rbf", C=C, gamma=gamma)),
# Criação e treino do modelo SVM com kernel especificado
        ("NuSVC", "linear", dict(kernel="linear", nu=nu)),
# Criação e treino do modelo SVM com kernel especificado
        ("NuSVC", "poly",   dict(kernel="poly", degree=degree, nu=nu, gamma=gamma)),
# Criação e treino do modelo SVM com kernel especificado
        ("NuSVC", "rbf",    dict(kernel="rbf", nu=nu, gamma=gamma)),
    ]
    for kind, kernel, params in cfgs:
# Criação e treino do modelo SVM com kernel especificado
        if kind == "SVC":
# Criação e treino do modelo SVM com kernel especificado
            clf = SVC(decision_function_shape="ovr", random_state=42, **params)
        else:
# Criação e treino do modelo SVM com kernel especificado
            clf = NuSVC(decision_function_shape="ovr", random_state=42, **params)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
# Cálculo da acurácia do modelo
        acc = accuracy_score(y_test, y_pred)
# Cálculo da acurácia do modelo
        bacc = balanced_accuracy_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        results.append({"model": kind, "kernel": kernel, "accuracy": acc, "balanced_accuracy": bacc})
        models[(kind, kernel)] = (clf, cm, y_pred)
    return pd.DataFrame(results), models

# Plotar a fronteira de decisão
def plot_boundary(model, X_train, X_test, y_test, title):
    X_vis = np.vstack([X_train, X_test])
    h = 0.02
    x_min, x_max = X_vis[:, 0].min() - 0.5, X_vis[:, 0].max() + 0.5
    y_min, y_max = X_vis[:, 1].min() - 0.5, X_vis[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid).reshape(xx.shape)
    plt.figure(figsize=(6,5))
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, s=20, edgecolor='k')
    plt.title(title)
    plt.xlabel("Feature 1 (scaled)")
    plt.ylabel("Feature 2 (scaled)")
    plt.tight_layout()
    plt.show()

# Plotar a matriz de confusão
def plot_cm(cm, title):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(values_format='d')
    plt.title(title)
    plt.tight_layout()
    plt.show()

all_rows = []
stored_models = {}
for dname, (X, y) in datasets.items():
    Xtr, Xte, ytr, yte = split_scale(X, y)
    df, models = eval_models(Xtr, Xte, ytr, yte, C=1.0, nu=0.3, degree=2, gamma="scale")
    df.insert(0, "dataset", dname)
    all_rows.append(df)
    stored_models[dname] = (Xtr, Xte, ytr, yte, models)

metrics = pd.concat(all_rows, ignore_index=True)
metrics.round(4)

for dname, (Xtr, Xte, ytr, yte, models) in stored_models.items():
    for kernel in ["linear", "poly", "rbf"]:
# Criação e treino do modelo SVM com kernel especificado
        clf, cm, _ = models[("SVC", kernel)]
# Criação e treino do modelo SVM com kernel especificado
        plot_boundary(clf, Xtr, Xte, yte, f"{dname} — Fronteira SVC ({kernel})")
# Criação e treino do modelo SVM com kernel especificado
        plot_cm(cm, f"{dname} — Matriz de Confusão SVC ({kernel})")
print(f'Acurácia: {acc:.4f} | Balanced Accuracy: {bacc:.4f}')